# Binary Search

Author: Isaac Flath

This is a binary search problem.  Given a rectangular grid and a start location, I must find the end location.

While I do not know the end location, after every step I will be given a direction (ie. UL, DR, L, etc.)

This problem is from codingame

In [26]:
#require "jupyter.notebook" ;;
(* #require "jupyter-archimedes" *)


#### Problem Setup

The first thing I need to do is calculate the direction of the target.  We can do that with a compare and match setup pretty easily. 

In [3]:
exception InvalidInput of string

let get_width_dir bw cw = 
  match compare cw bw  with
    |0 -> ""
    |1 -> "L"
    |(-1) -> "R"
    | _ -> raise (InvalidInput "error")

let get_height_dir bh ch = 
  match compare ch bh with
    |0 -> "" 
    |1 -> "U" 
    |(-1) -> "D"
    | _ -> raise (InvalidInput "error")

let get_direction loc target = 
  let bw,bh = target in
  let ((cw,ch),(_,_,_,_)) = loc in

  (get_height_dir bh ch) ^ (get_width_dir bw cw)

exception InvalidInput of string


val get_width_dir : 'a -> 'a -> string = <fun>


val get_height_dir : 'a -> 'a -> string = <fun>


val get_direction : ('a * 'b) * ('c * 'd * 'e * 'f) -> 'a * 'b -> string =
  <fun>


#### Step Function

Next we need to write a function to choose our next step.

For this first approach, I follow this 2 step process:
1. Based on my location and direction of target, shrink the rectangle of possible target cells
1. Select location in the middle of the location

This ensures I get the most information possible with each step.

In [5]:
let take_step1 loc direction =
  let ((cw,ch),(w0,w1,h0,h1)) = loc in
  match direction with 
        |"L" -> (((w0+cw)/2,ch),(w0,cw,ch,ch))
        |"R" -> (((cw+w1)/2,ch),(cw,w1,ch,ch))
        |"U" -> ((cw,(h0+ch)/2),(cw,cw,h0,ch))
        |"D" -> ((cw,(ch+h1)/2),(cw,cw,ch,h1))
        |"UL" -> (((w0+cw)/2,(h0+ch)/2),(w0,cw,h0,ch))
        |"UR" -> (((cw+w1)/2,(w0+ch)/2),(cw,w1,h0,ch))
        |"DL" -> (((w0+cw)/2,(ch+h1)/2),(w0,cw,ch,h1))
        |"DR" -> (((cw+w1)/2,(ch+h1)/2),(cw,w1,ch,h1))
        |_ -> raise (InvalidInput "error")

val take_step1 :
  (int * int) * (int * int * int * int) ->
  string -> (int * int) * (int * int * int * int) = <fun>


#### The Test Loop

Next we need a test loop (recursion) that wil continue to run until I find the target location.

First I define a function that will print out information.  This is helpful in seeing what it's doing at each step and debugging.

In [6]:
let print_step target loc direction =
    let ((ow,oh),(bw1,bw2,bh1,bh2)) = loc in
    let bw, bh = target in
    print_endline ("Board: " ^ string_of_int bw1 ^ "x"^ string_of_int bw2 ^ "x"^ string_of_int bh1 ^ "x"^ string_of_int bh2 ^ "  |  " ^
              "Bomb: " ^ string_of_int bw ^ "x" ^ string_of_int bh ^ "  |  " ^ 
              "Loc: " ^ string_of_int ow ^ "x" ^ string_of_int oh ^ "  |  " ^ 
              "Direction: " ^ direction);

val print_step :
  int * int -> (int * int) * (int * int * int * int) -> string -> unit =
  <fun>


In [11]:
let rec loop loc printf target =
    let direction = get_direction loc target in
    printf target loc direction;

    let out = take_step1 loc direction in
    let ((ow,oh),(tw1,tw2,th1,th2)) = out in


    if (ow,oh)=(target) then "success" 
    else loop out printf target

val loop :
  (int * int) * (int * int * int * int) ->
  (int * int -> (int * int) * (int * int * int * int) -> string -> 'a) ->
  int * int -> string = <fun>


#### The Expirament

We can see that even when run with a pretty large random size and locations setup, we solve the problem in relatively few steps.

In [17]:
let max_sz = 1_000_000
let w,h = (Random.int max_sz,Random.int max_sz)
let current_loc = (Random.int w,Random.int h)
let bomb_loc = (Random.int w,Random.int h)
let board = (0,w,0,h)
let current_loc = (current_loc,board)

val max_sz : int = 1000000


val w : int = 192869
val h : int = 233428


val current_loc : int * int = (71467, 101453)


val bomb_loc : int * int = (69195, 180906)


val board : int * int * int * int = (0, 192869, 0, 233428)


val current_loc : (int * int) * (int * int * int * int) =
  ((71467, 101453), (0, 192869, 0, 233428))


In [16]:
loop current_loc print_step bomb_loc

Board: 0x698678x0x695949  |  Bomb: 155526x598617  |  Loc: 675006x606536  |  Direction: UL
Board: 0x675006x0x606536  |  Bomb: 155526x598617  |  Loc: 337503x303268  |  Direction: DL
Board: 0x337503x303268x606536  |  Bomb: 155526x598617  |  Loc: 168751x454902  |  Direction: DL
Board: 0x168751x454902x606536  |  Bomb: 155526x598617  |  Loc: 84375x530719  |  Direction: DR
Board: 84375x168751x530719x606536  |  Bomb: 155526x598617  |  Loc: 126563x568627  |  Direction: DR
Board: 126563x168751x568627x606536  |  Bomb: 155526x598617  |  Loc: 147657x587581  |  Direction: DR
Board: 147657x168751x587581x606536  |  Bomb: 155526x598617  |  Loc: 158204x597058  |  Direction: DL
Board: 147657x158204x597058x606536  |  Bomb: 155526x598617  |  Loc: 152930x601797  |  Direction: UR
Board: 152930x158204x597058x601797  |  Bomb: 155526x598617  |  Loc: 155567x374727  |  Direction: DL
Board: 152930x155567x374727x601797  |  Bomb: 155526x598617  |  Loc: 154248x488262  |  Direction: DR
Board: 154248x155567x488262x6017

- : string = "success"
